In [818]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "06" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 18
repetitionName = "udrlBC2CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

35
3.888888888888889


In [819]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 153)
(10, 153)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.6625,0.6875,0.65,0.70625,0.7125,0.6875,0.66875,0.7,0.66875,0.70625,0.7125,0.69375,0.68125,0.66875,0.65625,0.7125,0.7125,0.6875,0.63125,0.6625,0.6375,0.6875,0.70625,0.69375,0.63125,0.64375,0.66875,0.71875,0.725,0.7,0.68125,0.68125,0.6625,0.71875,0.7125,0.6875,0.65625,0.68125,0.6625,0.725,0.7125,0.6875,0.65,0.65625,0.65625,0.7,0.7,0.68125,0.7375,0.71875,0.69375,0.71875,0.71875,0.70625,0.675,0.65625,0.65,0.66875,0.66875,0.65625,0.675,0.6875,0.6625,0.6625,0.6375,0.66875,0.6875,0.675,0.66875,0.68125,0.675,0.65625,0.675,0.6625,0.65625,0.60625,0.6125,0.64375,0.64375,0.6375,0.65625,0.6625,0.68125,0.65625,0.68125,0.6875,0.65625,0.69375,0.69375,0.69375,0.6625,0.66875,0.65,0.64375,0.65625,0.625,0.6625,0.66875,0.65625,0.625,0.625,0.6375,0.66875,0.675,0.69375,0.7375,0.725,0.70625,0.69375,0.70625,0.6875,0.725,0.71875,0.6875,0.7125,0.71875,0.70625,0.71875,0.71875,0.7,0.70625,0.7125,0.6875,0.70625,0.7125,0.6875,0.7,0.6875,0.69375,0.70625,0.71875,0.69375,0.70625,0.7125,0.69375,0.70625,0.7125,0.6875,0.675,0.6875,0.70625,0.6625,0.69375,0.6375,0.7,0.7125,0.6875,0.725,0.725,0.70625,0.6875,0.7,0.7
2,0.765625,0.760417,0.734375,0.71875,0.739583,0.729167,0.760417,0.755208,0.75,0.71875,0.739583,0.729167,0.78125,0.734375,0.729167,0.713542,0.734375,0.723958,0.734375,0.75,0.713542,0.723958,0.755208,0.71875,0.708333,0.71875,0.692708,0.692708,0.739583,0.723958,0.765625,0.770833,0.734375,0.713542,0.744792,0.729167,0.786458,0.75,0.75,0.708333,0.739583,0.729167,0.765625,0.760417,0.744792,0.703125,0.75,0.729167,0.75,0.765625,0.71875,0.739583,0.776042,0.71875,0.71875,0.723958,0.703125,0.723958,0.755208,0.729167,0.776042,0.755208,0.734375,0.75,0.75,0.71875,0.770833,0.75,0.734375,0.760417,0.760417,0.734375,0.755208,0.78125,0.755208,0.671875,0.708333,0.645833,0.697917,0.708333,0.6875,0.760417,0.765625,0.734375,0.765625,0.770833,0.739583,0.744792,0.744792,0.729167,0.755208,0.744792,0.723958,0.671875,0.682292,0.661458,0.734375,0.755208,0.723958,0.666667,0.692708,0.6875,0.78125,0.817708,0.744792,0.734375,0.739583,0.703125,0.703125,0.734375,0.723958,0.71875,0.739583,0.729167,0.713542,0.744792,0.723958,0.713542,0.713542,0.713542,0.734375,0.723958,0.729167,0.734375,0.739583,0.734375,0.671875,0.71875,0.703125,0.703125,0.729167,0.71875,0.723958,0.744792,0.729167,0.708333,0.739583,0.729167,0.729167,0.744792,0.713542,0.692708,0.682292,0.666667,0.708333,0.734375,0.723958,0.697917,0.71875,0.708333,0.755208,0.765625,0.75
3,0.6875,0.701389,0.694444,0.652778,0.680556,0.680556,0.729167,0.722222,0.6875,0.659722,0.680556,0.680556,0.701389,0.694444,0.694444,0.652778,0.680556,0.680556,0.708333,0.729167,0.6875,0.645833,0.659722,0.666667,0.680556,0.715278,0.6875,0.652778,0.680556,0.680556,0.701389,0.701389,0.701389,0.659722,0.6875,0.6805

In [820]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.82 : subject 2, feature:dataGCV-BCcn3--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.8 : subject 5, feature:welchDatacn3--welchDataBC, kernellinear, C:0.5
Accuracy 0.79 : subject 4, feature:fftDatacn3BC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 2, feature:welchDataBC--dataWCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 4, feature:welchDatacn3BC--fftDatacn3BC, kernellinear, C:0.5
Accuracy 0.78 : subject 4, feature:fftDatacn3BC--welchDataBC, kernellinear, C:0.5
Accuracy 0.78 : subject 2, feature:welchDatacn3--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 2, feature:welchDataBC--welchData, kernellinear, C:0.5
Accuracy 0.78 : subject 2, feature:dataGCV-BCcn3--welchDataBC, kernellinear, C:0.5
Accuracy 0.78 : subject 2, feature:fftDatacn3BC--fftDataBC, kernelsigmoid, C:2.5


In [821]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.82 : subject 2, feature:dataGCV-BCcn3--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 2, feature:welchDataBC--dataWCV-BC, kernellinear, C:0.5

Accuracy 0.8 : subject 5, feature:welchDatacn3--welchDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:welchDataBC--dataHR, kernellinear, C:0.5

Accuracy 0.79 : subject 4, feature:fftDatacn3BC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 4, feature:fftDatacn3BC--welchDataBC, kernellinear, C:0.5

Accuracy 0.75 : subject 7, feature:dataHRcn3--welchDataBC, kernelrbf, C:2.5
Accuracy 0.74 : subject 7, feature:dataHRcn3--welchDataBC, kernellinear, C:0.5

Accuracy 0.75 : subject 8, feature:fftDatacn3BC--dataGCV-BC, kernelrbf, C:2.5
Accuracy 0.74 : subject 8, feature:fftDatacn3BC--fftDataBC, kernellinear, C:0.5

Accuracy 0.74 : subject 1, feature:fftDatacn3BC--dataHRBC, kernellinear, C:0.5
Accuracy 0.74 : subject 1, feature:welchDataBC--fftDataBC, kernellinear, C:0.5

Accuracy 0.

In [822]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.68312908 0.73103894 0.67696986 0.72092865 0.72314134
 0.6761735  0.70050381 0.70829248 0.66072077]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152
0,0.705889,0.700859,0.693157,0.698173,0.700754,0.704374,0.712237,0.697934,0.693312,0.700473,0.701473,0.705777,0.709014,0.690365,0.686237,0.697257,0.698464,0.702739,0.694669,0.69517,0.681513,0.699716,0.700624,0.699614,0.692603,0.700821,0.687328,0.698839,0.705952,0.703241,0.706804,0.701708,0.691972,0.700256,0.70141,0.703742,0.705854,0.700165,0.695072,0.696142,0.700638,0.704374,0.703984,0.691986,0.694031,0.696307,0.696728,0.702059,0.719778,0.710722,0.700989,0.706183,0.718406,0.713496,0.685792,0.687247,0.669178,0.693301,0.696654,0.687805,0.707769,0.700807,0.693336,0.686188,0.690323,0.695241,0.70685,0.685809,0.691828,0.704075,0.69826,0.690467,0.69354,0.700589,0.685799,0.686013,0.697313,0.683168,0.687935,0.698004,0.681064,0.704216,0.697938,0.691625,0.707884,0.703805,0.694974,0.713412,0.722776,0.706092,0.712104,0.712181,0.696489,0.657684,0.681997,0.661707,0.696451,0.697373,0.686069,0.684568,0.681902,0.68656,0.704167,0.690351,0.685192,0.705159,0.703855,0.687984,0.692052,0.703507,0.700747,0.700039,0.701975,0.704374,0.698751,0.70424,0.703356,0.702104,0.700586,0.707741,0.699032,0.697422,0.704451,0.70134,0.700238,0.703486,0.695076,0.698173,0.695072,0.701014,0.702557,0.703511,0.698131,0.700638,0.704833,0.701789,0.700561,0.704374,0.708375,0.700796,0.708207,0.676975,0.680657,0.680001,0.69529,0.704949,0.700747,0.703665,0.698222,0.699172,0.701203,0.705717,0.69987


Max average accuracy 0.7227763748597082


In [823]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

fftDatacn3BC--welchDataBC
sigmoid
0.7227763748597082

welchDataBC--fftDataBC
linear
0.7197776374859708

fftDatacn3BC--fftDataBC
sigmoid
0.7184062850729518

fftDatacn3BC--fftDataBC
rbf
0.7134960718294051

fftDatacn3BC--welchDataBC
linear
0.7134118967452302



In [824]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-welchDataBC
1
-fftDatacn3BC
2
welchDataBC--fftData
3
fftDatacn3BC--fftData
4
welchDataBC--welchData
5
fftDatacn3BC--welchData
6
welchDataBC--dataHR
7
fftDatacn3BC--dataHR
8
welchDataBC--dataCorr1d
9
fftDatacn3BC--dataCorr1d
10
welchDataBC--dataFFTCV-BC
11
fftDatacn3BC--dataFFTCV-BC
12
welchDataBC--dataWCV-BC
13
fftDatacn3BC--dataWCV-BC
14
welchDataBC--dataHRCV-BC
15
fftDatacn3BC--dataHRCV-BC
16
welchDataBC--fftDataBC
17
fftDatacn3BC--fftDataBC
18
dataHRBC--welchDataBC
19
gaussianData--welchDataBC
20
dataGCVBC--welchDataBC
21
gaussianDataBC--welchDataBC
22
dataGCV-BC--welchDataBC
23
fftDatacn3--welchDataBC
24
welchDatacn3--welchDataBC
25
dataHRcn3--welchDataBC
26
dataCorr1dcn3--welchDataBC
27
dataFFTCV-BCcn3--welchDataBC
28
dataWCV-BCcn3--welchDataBC
29
fftDatacn3BC--welchDataBC
30
welchDatacn3BC--welchDataBC
31
dataHRcn3BC--welchDataBC
32
gaussianDatacn3--welchDataBC
33
gaussianDatacn3BC--welchDataBC
34
dataGCV-BCcn3--welchDataBC
35
fftDatacn3BC--dataHRBC
36
fftDatacn3BC--gaussianData

In [825]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152
0,6.352999,6.307734,6.238415,6.283554,6.306787,6.339362,6.410133,6.281408,6.239804,6.304261,6.313258,6.351989,6.381124,6.213289,6.176136,6.275316,6.286174,6.324653,6.25202,6.256534,6.133617,6.297443,6.305619,6.296528,6.233428,6.307386,6.185953,6.289552,6.353567,6.329167,6.361237,6.315372,6.227746,6.302304,6.312689,6.333681,6.352683,6.301484,6.25565,6.265278,6.305745,6.339362,6.335859,6.227872,6.246275,6.266761,6.270549,6.318529,6.477999,6.396496,6.308902,6.35565,6.465657,6.421465,6.172128,6.185227,6.022601,6.23971,6.269886,6.190246,6.369918,6.30726,6.240025,6.175694,6.21291,6.257165,6.361648,6.172285,6.226452,6.336679,6.284343,6.214205,6.241856,6.305303,6.172191,6.174116,6.275821,6.148516,6.191414,6.282039,6.129577,6.337942,6.281439,6.224621,6.37096,6.334249,6.254766,6.420707,6.504987,6.35483,6.408933,6.409628,6.268403,5.91916,6.137973,5.955366,6.268056,6.276357,6.174621,6.161111,6.137121,6.17904,6.3375,6.213163,6.16673,6.346433,6.334691,6.191856,6.228472,6.331566,6.306723,6.300347,6.317771,6.339362,6.288763,6.338163,6.330208,6.318939,6.305271,6.369665,6.291288,6.276799,6.340057,6.312058,6.302146,6.331376,6.255682,6.283554,6.25565,6.309122,6.323011,6.331597,6.283176,6.305745,6.343497,6.316098,6.305051,6.339362,6.375379,6.307165,6.373864,6.092771,6.125915,6.120013,6.257607,6.344539,6.306723,6.332986,6.283996,6.292551,6.310827,6.351452,6.298832


88
9.0
Max total significant accuracy then averaged 0.7227763748597082 
fftDatacn3BC--welchDataBC
sigmoid
C: 2.5 


In [826]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
